In [1]:
from importlib import reload

In [2]:
import load_data

Loading up the load data and exploring the columns

In [4]:
loans = load_data.read_zipped_csv('./lending-club-loan-data.zip', 'loan.csv')
print(loans.columns)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

Now, create a table model with all available columns.

In [9]:
import table_model
reload(table_model)

feature_columns = {
    'loan_amnt': table_model.NumericColumn(),
    'funded_amnt': table_model.NumericColumn(),
    'funded_amnt_inv': table_model.NumericColumn(),
    'term': table_model.CategoricalColumn(),
    'int_rate': table_model.PercentageColumn(),
    'installment': table_model.NumericColumn(),
    'grade': table_model.CategoricalColumn(),
    'sub_grade': table_model.CategoricalColumn(),
    'emp_length': table_model.CategoricalColumn(),
    'addr_state': table_model.CategoricalColumn(),
    'home_ownership': table_model.CategoricalColumn(),
    'annual_inc': table_model.NumericColumn(),
    'annual_inc_joint': table_model.NumericColumn(),
    'dti': table_model.NumericColumn(),
    'dti_joint': table_model.NumericColumn(),
    'open_acc': table_model.NumericColumn(),
    'total_acc': table_model.NumericColumn(),
    'pub_rec': table_model.NumericColumn(),
    'tot_coll_amt': table_model.NumericColumn(),
    'total_bal_il': table_model.NumericColumn(),
    'tot_cur_bal': table_model.NumericColumn(),
    'revol_bal': table_model.NumericColumn(),
    'revol_util': table_model.PercentageColumn(),
    'mths_since_last_delinq': table_model.NumericColumn(),
    'mths_since_last_record': table_model.NumericColumn(),
    'mths_since_last_major_derog': table_model.NumericColumn(),
    'inq_fi': table_model.NumericColumn(),
    'inq_last_12m': table_model.NumericColumn(),
    'all_util': table_model.NumericColumn(),
    'acc_now_delinq': table_model.NumericColumn(),
    # and the output
    'loan_status': table_model.OutputLabelColumn(),
}

loan_model = table_model.TableModel(feature_columns, 'loan_status')
filtered = loans.loc[loans.loan_status.isin(['Current', 'Default'])]
loan_model.fit(filtered)
x, y = loan_model.transform(filtered)

In [12]:
print(x, x.dtype)

[[-1.44078183 -1.44083977 -1.44067287 ..., -0.07797889 -0.14169542
  -0.06851428]
 [-0.9700067  -0.96998012 -0.96960402 ..., -0.07797889 -0.14169542
  -0.06851428]
 [-0.61692536 -0.6168353  -0.6192466  ..., -0.07797889 -0.14169542
  -0.06851428]
 ..., 
 [-0.26384395 -0.26369056 -0.26300082 ..., -0.07797889 -0.14169542
  -0.06851428]
 [-0.38153774 -0.38140547 -0.380768   ..., -0.07797889 -0.14169542
  -0.06851428]
 [ 0.56001258  0.56031394  0.5613696  ..., -0.07797889 -0.14169542
  -0.06851428]] float32


In [13]:
print(y, y.dtype)

[0 0 0 ..., 0 0 0] int32


In [14]:
import keras

keras.backend.clear_session()

model = keras.models.Sequential()
y_one_hot = keras.utils.to_categorical(y)
# logistic regression is a one layer model
model.add(keras.layers.Dense(y_one_hot.shape[1], activation='sigmoid', input_dim=x.shape[1]))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y_one_hot, epochs=4)

Epoch 1/4
602998/602998 [==============================] - 18s 31us/step - loss: 0.0245
Epoch 2/4
602998/602998 [==============================] - 17s 29us/step - loss: 0.0141
Epoch 3/4
602998/602998 [==============================] - 17s 29us/step - loss: 0.0141
Epoch 4/4
602998/602998 [==============================] - 17s 29us/step - loss: 0.0141


In [15]:
predictions = model.predict_classes(x)

In [16]:
import sklearn.metrics
print(sklearn.metrics.accuracy_score(y, predictions))

0.997978434423


Looks good -- but there is probably a catch here, looking at the balance of classes, far more loans are 'Current' than 'Default', so our model probaly just learned to guess 'Current'.

In [17]:
import numpy as np
labels, counts = np.unique(y, return_counts=True)
print(labels, counts)

[0 1] [601779   1219]


In [18]:
print(1 - (counts[1] / counts[0]))

0.997974339417


So -- these highly imbalanced classes will make it hard to predict 'Default', let's see it that was the case.

In [19]:
print(np.unique(predictions, return_counts=True))

(array([0]), array([602998]))


In [20]:
print(sklearn.metrics.classification_report(y, predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      1.00      1.00    601779
    Default       0.00      0.00      0.00      1219

avg / total       1.00      1.00      1.00    602998



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


So -- clearly the model isn't actually very useful, even though it is accurate! 
Real data often needs to predict relatively rare events.

In [21]:
reload(table_model)

logistic_regression_model = table_model.KerasLogisticRegressionModel()
logistic_regression_model.fit(x, y)

Epoch 1/16
602998/602998 [==============================] - 15s 25us/step - loss: 1.3307
Epoch 2/16
602998/602998 [==============================] - 16s 27us/step - loss: 1.2721
Epoch 3/16
602998/602998 [==============================] - 16s 27us/step - loss: 1.2776
Epoch 4/16
602998/602998 [==============================] - 18s 31us/step - loss: 1.2654
Epoch 5/16
602998/602998 [==============================] - 16s 26us/step - loss: 1.2714
Epoch 6/16
602998/602998 [==============================] - 16s 26us/step - loss: 1.2662


KerasLogisticRegressionModel(epochs=16, verbose=1)

In [22]:
balanced_predictions = logistic_regression_model.predict(x)

In [23]:
print(sklearn.metrics.accuracy_score(y, balanced_predictions))

0.637788185035


In [24]:
print(np.unique(balanced_predictions, return_counts=True))

(array([0, 1]), array([384144, 218854]))


So - the accuracy went down, but the model is at leasty trying to predict 'Default'!

In [25]:
print(sklearn.metrics.classification_report(y, balanced_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.64      0.78    601779
    Default       0.00      0.68      0.01      1219

avg / total       1.00      0.64      0.78    602998



Now - to try a deep learning model.

In [26]:
reload(table_model)
deep_model = table_model.KerasDeepClassifierModel()
deep_model.fit(x, y)

Epoch 1/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.3882
Epoch 2/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.2841
Epoch 3/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.2616
Epoch 4/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.2585
Epoch 5/16
602998/602998 [==============================] - 30s 51us/step - loss: 1.2332
Epoch 6/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.2385
Epoch 7/16
602998/602998 [==============================] - 31s 52us/step - loss: 1.2280
Epoch 8/16
602998/602998 [==============================] - 30s 51us/step - loss: 1.2177
Epoch 9/16
602998/602998 [==============================] - 30s 50us/step - loss: 1.2093
Epoch 10/16
602998/602998 [==============================] - 31s 51us/step - loss: 1.2043
Epoch 11/16
602998/602998 [==============================] - 30s 50us/step - loss: 1.2105
Epoch 12/16
602998/

KerasDeepClassifierModel(epochs=16, verbose=1)

In [27]:
deep_predictions = deep_model.predict(x)

In [28]:
print(sklearn.metrics.accuracy_score(y, deep_predictions))

0.763611156256


In [29]:
print(np.unique(deep_predictions, return_counts=True))

(array([0, 1]), array([460073, 142925]))


In [30]:
print(sklearn.metrics.classification_report(y, deep_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.76      0.87    601779
    Default       0.01      0.66      0.01      1219

avg / total       1.00      0.76      0.86    602998



And now -- combining both of these techniques in Wide + Deep models.

In [47]:
reload(table_model)
deep_model = table_model.KerasWideAndDeepClassifierModel()
deep_model.fit(x, y)

Epoch 1/16
602998/602998 [==============================] - 37s 62us/step - loss: 1.3195
Epoch 2/16
602998/602998 [==============================] - 36s 60us/step - loss: 1.2838
Epoch 3/16
602998/602998 [==============================] - 38s 63us/step - loss: 1.2687
Epoch 4/16
602998/602998 [==============================] - 42s 70us/step - loss: 1.2660
Epoch 5/16
602998/602998 [==============================] - 46s 76us/step - loss: 1.2621
Epoch 6/16
602998/602998 [==============================] - 48s 80us/step - loss: 1.2567
Epoch 7/16
602998/602998 [==============================] - 47s 77us/step - loss: 1.2581
Epoch 8/16
602998/602998 [==============================] - 49s 81us/step - loss: 1.2555
Epoch 9/16
602998/602998 [==============================] - 49s 81us/step - loss: 1.2569
Epoch 10/16
602998/602998 [==============================] - 40s 66us/step - loss: 1.2498
Epoch 11/16
602998/602998 [==============================] - 34s 56us/step - loss: 1.2475
Epoch 12/16
602998/

KerasWideAndDeepClassifierModel(epochs=16, verbose=1)

In [48]:
wd_predictions = deep_model.predict(x)

In [49]:
print(sklearn.metrics.accuracy_score(y, wd_predictions))

0.608940659836


In [50]:
print(np.unique(wd_predictions, return_counts=True))

(array([0, 1]), array([366543, 236455]))


In [51]:
print(sklearn.metrics.classification_report(y, wd_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.61      0.76    601779
    Default       0.00      0.77      0.01      1219

avg / total       1.00      0.61      0.75    602998

